In [1]:
import collections
import numpy as np
import csv
import os
import nltk
import re

list_of_characters=[]
charact_vocab = collections.defaultdict(list)
tagged_vocab = collections.defaultdict(list)
charact_sents = collections.defaultdict(list)
charact_attrs = collections.defaultdict(list)
charact_mods = collections.defaultdict(list)
attrsets = collections.defaultdict(list)
modsets = collections.defaultdict(list)
all_tags = set()

for subdir, dirs, files in os.walk('/home/eshaw/Documents/NLP/eshaw2-finalproject/txt'):
    print len(files)
    for file in files:
        character_name = file[0:-15]
        list_of_characters.append(character_name)
        file_path = subdir + os.path.sep + file
        shakes = open(file_path, 'r')
        corpus = shakes.read()
        charact_vocab[character_name] = corpus.split(" ")
        tagged_vocab[character_name] = nltk.pos_tag(corpus.split(" "))
        charact_sents[character_name] = re.split('[?!;.:]',corpus)
        del charact_sents[character_name][-1]
        
        attrs = []
        mods = []
        for pair in tagged_vocab[character_name]:
            word, tag = pair
#             all_tags.add(tag)
            if re.search('JJ', tag) or re.search('NN', tag):
                attrs.append(word)
            if re.search('JJ', tag) or re.search('RB', tag):
                mods.append(word)
        charact_attrs[character_name] = attrs
        attrsets[character_name] = set(attrs)
        charact_mods[character_name] = mods
        modsets[character_name] = set(mods)
# print charact_vocab['BOTTOM']

19


In [2]:
import jsonrpc
from simplejson import loads
server = jsonrpc.ServerProxy(jsonrpc.JsonRpc20(),jsonrpc.TransportTcpIp(addr=("127.0.0.1", 8080)))

In [7]:
mod_types = collections.defaultdict(list)
attr_tags = set(['nsubj','appos','amod','nn'])
mod_tags = set(['acomp','advmod','prepc','tmod','xcomp'])
# tags = set()

char_dep = np.zeros((len(list_of_characters),2))
# char_dep = collections.defaultdict(list)
for c,character in enumerate(list_of_characters):
    print character
#     attr_count = 0
#     mod_count = 0
    cur_attrset = attrsets[character]
    cur_modset = modsets[character]
    for s, sentence in enumerate(charact_sents[character]):
#         print s
        result = loads(server.parse(sentence))
        for relation in result['sentences'][0]['dependencies']:
            if (relation[1] in cur_attrset) or (relation[2] in cur_attrset):
                if relation[0] in attr_tags:
                    char_dep[c,0] += 1
#                     attr_count += 1
            if (relation[1] in cur_modset) or (relation[2] in cur_modset):
                if relation[0] in mod_tags:
                    char_dep[c,1] += 1
#                     mod_count += 1
    #             tags.update([relation[0]])
#     mod_types[c:] = {'Attributes': attr_count,'Modifiers': mod_count}

# verb_types

TITANIA
FLUTE
BOTTOM
Wall
OBERON
PHILOSTRATE
EGEUS
HELENA
Thisbe
QUINCE
Pyramus
HERMIA
THESEUS
LYSANDER
Prologue
PUCK
HIPPOLYTA
DEMETRIUS
Fairy


In [8]:
char_dep

array([[ 195.,   44.],
       [  30.,    9.],
       [ 243.,   65.],
       [  16.,   10.],
       [ 255.,   77.],
       [  34.,   10.],
       [  53.,    5.],
       [ 253.,  109.],
       [  38.,    4.],
       [  82.,   23.],
       [  78.,   22.],
       [ 182.,  104.],
       [ 237.,   69.],
       [ 180.,   88.],
       [  50.,    9.],
       [ 204.,   85.],
       [  35.,   10.],
       [ 155.,   59.],
       [  27.,    8.]])

In [10]:
np.savetxt('/home/eshaw/Documents/NLP/eshaw2-finalproject/dream_mod_types.txt', char_dep)

In [6]:
# saved_verb_types = mod_types

with open('/home/eshaw/Documents/NLP/eshaw2-finalproject/dream_mod_types.csv', 'wb') as f:
    w = csv.DictWriter(f, mod_types.keys())
    w.writeheader()
    w.writerow(mod_types)